# Kanji Radical Match

Data processing, training, and output.

In [ ]:
import utils as u
from utils import KanjiFFNN
import torch.optim as optim
import torch.nn as nn

e2r_model = KanjiFFNN()
criterion = nn.BCELoss()

optimizer = optim.SGD(e2r_model.parameters(), lr=0.01)